In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
nltk.download("punkt")

Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anmoljeet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
def clean(sentences):
    wordList = []
    for i in sentences:
        subbedSentence = re.sub(r'[^ a-z A-Z 0-9]', " ", i)
        words = word_tokenize(subbedSentence)
        wordList.append([j.lower() for j in words]) 
    return wordList

def tokens(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    token = Tokenizer(filters = filters)
    token.fit_on_texts(words)
    return token

def maxLength(words):
    return(len(max(words, key = len)))

def encoder(tokens, words):
    return(tokens.texts_to_sequences(words))

def padder(words, maxLen):
    return(pad_sequences(words, maxlen = maxLengthz, padding = "post"))

def oneHotEncoder(encode):
    oneHot = OneHotEncoder(sparse = False)
    return(oneHot.fit_transform(encode))


def create_model(vocabSizez):

    model = Sequential()
    model.add(Embedding(vocabSizez, 128))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation = "softmax"))
  
    return model

def predict(text):
    cleanedSentence = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    testWord = word_tokenize(cleanedSentence)
    testWord = [w.lower() for w in testWord]
    testTokens = wordTokens.texts_to_sequences(testWord)
    print(testWord)
    if [] in testTokens: #Check for unknown words
        testTokens = list(filter(None, testTokens))
    testTokens = np.array(testTokens).reshape(1, len(testTokens))
    x = padder(testTokens, maxLengthz)
    pred = model.predict_proba(x)
  
    return pred

def getIntent(predicts, intents):
    prediction = predicts[0]
    intents = np.array(intents)
    ids = np.argsort(-prediction)
    intents = intents[ids]
    predictions = -np.sort(-prediction)
    for i in range(predicts.shape[1]):
        print("%s has confidence = %s" % (intents[i], (predictions[i])))
    result = np.where(predictions == np.amax(predictions))
    return intents[result]

In [29]:
dataSet = pd.read_csv('merged.csv', encoding = "latin1", names = ["Sentence", "Intent"])

dataSet, test = train_test_split(dataSet, shuffle = True, test_size=0.1)

intent = dataSet["Intent"]
keyIntent = list(set(intent))
sentences = list(dataSet["Sentence"])
  
cleanedSetences = clean(sentences)

wordTokens = tokens(cleanedSetences)

vocabSize = len(wordTokens.word_index) + 1
maxLengthz = maxLength(cleanedSetences)

encodedSentences = encoder(wordTokens, cleanedSetences)

paddedSentences = padder(encodedSentences, maxLengthz)

intentTokens = tokens(keyIntent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
tokenOutput = encoder(intentTokens, intent)
tokenOutput = np.array(tokenOutput).reshape(len(tokenOutput), 1)
oneHotOutput = oneHotEncoder(tokenOutput)

xTrain, xValid, yTrain, yValid = train_test_split(paddedSentences, oneHotOutput, shuffle = True, test_size = 0.11111111111111)



In [30]:
input_shape = xTrain.shape
#print(input_shape)
model = create_model(vocabSize)
model.summary()
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

modelFileName = 'modelLSTM.h5'
checkpoint = ModelCheckpoint(modelFileName, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(xTrain, yTrain, epochs = 150, batch_size = 32, validation_data = (xValid, yValid), callbacks = [checkpoint])

model.save(modelFileName)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         257792    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 165       
Total para

8000/8000 [==============================] - 4s 561us/step - loss: 0.0266 - acc: 0.9951 - val_loss: 2.0816e-06 - val_acc: 1.0000

Epoch 00036: val_loss did not improve from 0.00000
Epoch 37/150
8000/8000 [==============================] - 8s 953us/step - loss: 0.0065 - acc: 0.9971 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00037: val_loss did not improve from 0.00000
Epoch 38/150
8000/8000 [==============================] - 4s 544us/step - loss: 0.0044 - acc: 0.9985 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00038: val_loss did not improve from 0.00000
Epoch 39/150
8000/8000 [==============================] - 4s 551us/step - loss: 0.0053 - acc: 0.9979 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00039: val_loss did not improve from 0.00000
Epoch 40/150
8000/8000 [==============================] - 5s 668us/step - loss: 0.0044 - acc: 0.9981 - val_loss: 1.2004e-07 - val_acc: 1.0000

Epoch 00040: val_loss did not improve from 0.00000
Epoch 41/150
8000/8000 [===================

8000/8000 [==============================] - 5s 634us/step - loss: 0.0022 - acc: 0.9990 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00078: val_loss did not improve from 0.00000
Epoch 79/150
8000/8000 [==============================] - 5s 639us/step - loss: 0.0021 - acc: 0.9989 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00079: val_loss did not improve from 0.00000
Epoch 80/150
8000/8000 [==============================] - 7s 826us/step - loss: 0.0023 - acc: 0.9990 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00080: val_loss did not improve from 0.00000
Epoch 81/150
8000/8000 [==============================] - 6s 726us/step - loss: 0.0032 - acc: 0.9984 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00081: val_loss did not improve from 0.00000
Epoch 82/150
8000/8000 [==============================] - 5s 670us/step - loss: 0.0027 - acc: 0.9983 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00082: val_loss did not improve from 0.00000
Epoch 83/150
8000/8000 [===================

8000/8000 [==============================] - 5s 631us/step - loss: 0.0036 - acc: 0.9978 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00120: val_loss did not improve from 0.00000
Epoch 121/150
8000/8000 [==============================] - 7s 822us/step - loss: 0.0024 - acc: 0.9988 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00121: val_loss did not improve from 0.00000
Epoch 122/150
8000/8000 [==============================] - 6s 732us/step - loss: 0.0025 - acc: 0.9985 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00122: val_loss did not improve from 0.00000
Epoch 123/150
8000/8000 [==============================] - 5s 648us/step - loss: 0.0028 - acc: 0.9983 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00123: val_loss did not improve from 0.00000
Epoch 124/150
8000/8000 [==============================] - 5s 646us/step - loss: 0.0014 - acc: 0.9991 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00124: val_loss did not improve from 0.00000
Epoch 125/150
8000/8000 [==============

In [5]:
model = load_model('modelLSTM.h5')

In [49]:
text = "Create the cent"
pred = predict(text)
print(getIntent(pred, keyIntent))

['create', 'the', 'cent']
add_class has confidence = 1.0
add_attribute has confidence = 0.0
create_inheritance has confidence = 0.0
create_composition has confidence = 0.0
create_association has confidence = 0.0
['add_class']


In [60]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

complexTest = pd.read_csv('complexTestData.csv', encoding = "latin1", names = ["Sentence", "Intent"])

yPred = []
yTrue = []
#plug in test or complexTest
for i, j in complexTest.iterrows(): #i in index, j is value at row i
    pred = predict(j['Sentence'])
    intents = getIntent(pred, keyIntent)
    yPred.append(intents)
    yTrue.append(j['Intent'])
    
    
    


['create', 'a', 'school']
add_class has confidence = 1.0
add_attribute has confidence = 0.0
create_inheritance has confidence = 0.0
create_composition has confidence = 0.0
create_association has confidence = 0.0
['create', 'a', 'playing', 'card']
add_class has confidence = 1.0
add_attribute has confidence = 0.0
create_inheritance has confidence = 0.0
create_composition has confidence = 0.0
create_association has confidence = 0.0
['create', 'an', 'alumni']
add_class has confidence = 1.0
add_attribute has confidence = 0.0
create_inheritance has confidence = 0.0
create_composition has confidence = 0.0
create_association has confidence = 0.0
['make', 'a', 'lose', 'turn', 'action', 'card']
add_class has confidence = 1.0
add_attribute has confidence = 0.0
create_inheritance has confidence = 0.0
create_composition has confidence = 0.0
create_association has confidence = 0.0
['add', 'work', 'in', 'person']
create_composition has confidence = 1.0
create_association has confidence = 3.0052626e-3

In [61]:
print(accuracy_score(yTrue, yPred))
print(f1_score(yTrue, yPred, average='macro'))

0.5833333333333334
0.5091687979539642


C:\Users\anmoljeet\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
#test.to_csv (r'testData.csv', index = None, header = None)